#### License for [parts of the following code](https://github.com/farhanfuadabir/SHL-2020/blob/master/LICENSE)

MIT License

Copyright (c) 2020 Farhan Fuad

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the 'Software'), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED 'AS IS', WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

In [1]:
import pandas as pd

location = 'shl-dataset/train/Hand/'

acc_x = pd.read_csv(location + 'Acc_x.txt', header=None, sep=' ').to_numpy()
print('Acc_x Import Done')
acc_y = pd.read_csv(location + 'Acc_y.txt', header=None, sep=' ').to_numpy()
print('Acc_y Import Done')
acc_z = pd.read_csv(location + 'Acc_z.txt', header=None, sep=' ').to_numpy()
print('Acc_z Import Done')
mag_x = pd.read_csv(location + 'Mag_x.txt', header=None, sep=' ').to_numpy()
print('Mag_x Import Done')
mag_y = pd.read_csv(location + 'Mag_y.txt', header=None, sep=' ').to_numpy()
print('Mag_y Import Done')
mag_z = pd.read_csv(location + 'Mag_z.txt', header=None, sep=' ').to_numpy()
print('Mag_z Import Done')
gyr_x = pd.read_csv(location + 'Gyr_x.txt', header=None, sep=' ').to_numpy()
print('Gyr_x Import Done')
gyr_y = pd.read_csv(location + 'Gyr_y.txt', header=None, sep=' ').to_numpy()
print('Gyr_y Import Done')
gyr_z = pd.read_csv(location + 'Gyr_z.txt', header=None, sep=' ').to_numpy()
print('Gyr_z Import Done')

labels = pd.read_csv(location + 'Label.txt', header=None, sep=' ').mode(axis=1).to_numpy().flatten()
print('Label Import Done')

Acc_x Import Done
Acc_y Import Done
Acc_z Import Done
Mag_x Import Done
Mag_y Import Done
Mag_z Import Done
Gyr_x Import Done
Gyr_y Import Done
Gyr_z Import Done
Label Import Done


In [2]:
import numpy as np

# Select the acceleration magnitude as the main feature
acc = np.sqrt(acc_x**2 + acc_y**2 + acc_z**2)

print(acc.shape)

(196072, 500)


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = np.resize(acc, (acc.shape[0], 256)) # Use n samples for each label

scaler = StandardScaler()
X = scaler.fit_transform(X)

y = labels

X = X.reshape((X.shape[0], X.shape[1], 1))

idx = np.random.permutation(len(X))
X = X[idx]
y = y[idx]

print(X.shape, y.shape)

(196072, 256, 1) (196072,)


In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Check that we can use our GPU, to not wait forever during training
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 17812630953663744454,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 3215029044
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 11882337018855110940
 physical_device_desc: "device: 0, name: GeForce GTX 770, pci bus id: 0000:01:00.0, compute capability: 3.0"]

In [5]:
def make_model():
    input_layer = keras.layers.Input((256, 1))

    conv1 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(input_layer)
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.ReLU()(conv1)

    conv2 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv1)
    conv2 = keras.layers.BatchNormalization()(conv2)
    conv2 = keras.layers.ReLU()(conv2)

    conv3 = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(conv2)
    conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.ReLU()(conv3)

    gap = keras.layers.GlobalAveragePooling1D()(conv3)

    output_layer = keras.layers.Dense(9, activation="softmax")(gap)

    return keras.models.Model(inputs=input_layer, outputs=output_layer)

model = make_model()

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 1)            0         
_________________________________________________________________
conv1d (Conv1D)              (None, 256, 64)           256       
_________________________________________________________________
batch_normalization (BatchNo (None, 256, 64)           256       
_________________________________________________________________
re_lu (ReLU)                 (None, 256, 64)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 256, 64)           12352     
_________________________________________________________________
batch_normalization_1 (Batch (None, 256, 64)           256       
_________________________________________________________________
re_lu_1 (ReLU)               (None, 256, 64)           0         
__________

In [9]:
# Model training

epochs = 200
batch_size = 32

callbacks = [
    keras.callbacks.ModelCheckpoint(
        'shl-model.h5', save_best_only=True, monitor='val_loss'
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', factor=0.5, patience=20, min_lr=0.0001
    ),
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=50, verbose=1),
]

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy'],
)

history = model.fit(
    X, 
    y,
    epochs=epochs, 
    batch_size=batch_size, 
    callbacks=callbacks, 
    validation_split=0.2, 
    verbose=1
)

Train on 156857 samples, validate on 39215 samples
Epoch 1/200
156857/156857 [==============================] - 58s 369us/step - loss: 1.9537 - sparse_categorical_accuracy: 0.2275 - val_loss: 1.9534 - val_sparse_categorical_accuracy: 0.2277
Epoch 2/200
156857/156857 [==============================] - 56s 359us/step - loss: 1.9484 - sparse_categorical_accuracy: 0.2298 - val_loss: 1.9521 - val_sparse_categorical_accuracy: 0.2250
Epoch 3/200
156857/156857 [==============================] - 56s 357us/step - loss: 1.9463 - sparse_categorical_accuracy: 0.2310 - val_loss: 1.9464 - val_sparse_categorical_accuracy: 0.2291
Epoch 4/200
156857/156857 [==============================] - 56s 358us/step - loss: 1.9438 - sparse_categorical_accuracy: 0.2319 - val_loss: 1.9496 - val_sparse_categorical_accuracy: 0.2252
Epoch 5/200
156857/156857 [==============================] - 56s 358us/step - loss: 1.9412 - sparse_categorical_accuracy: 0.2340 - val_loss: 1.9448 - val_sparse_categorical_accuracy: 0.2356

KeyboardInterrupt: 